In [1]:
import pandas as pd
from pathlib import Path
from config import *
import psycopg2
import numpy as np

In [2]:
# Make a connection to our cloud PostgresSql database
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port= PORT)
cursor = conn.cursor()

In [3]:
# Read in results_weather dataframe. Bring in only the columns that we need. 
cursor.execute("Select * FROM results_weather")
data = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
colnames1 = ['raceId','resultId','driverId','constructorId','grid','position','year','round','weather_warm',
            'weather_cold','weather_dry','weather_wet','weather_cloudy']
results_weather_df = pd.DataFrame(data, columns=colnames)
results = results_weather_df[colnames1]
print(results.shape)
results.head()

(25040, 13)


,raceId,resultId,driverId,constructorId,grid,position,year,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,18,1,1,1,1,1,2008,1,1,0,0,0,0
1,18,2,2,2,5,2,2008,1,1,0,0,0,0
2,18,3,3,3,7,3,2008,1,1,0,0,0,0
3,18,4,4,4,11,4,2008,1,1,0,0,0,0
4,18,5,5,1,3,5,2008,1,1,0,0,0,0


In [4]:
# Read in races dataframe. Bring in only the columns that we need. 
cursor.execute("Select * FROM races")
data = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
colnames1 = ['raceId', 'circuitId']
races_df = pd.DataFrame(data, columns=colnames)
races= races_df[colnames1]
print(races.shape)
races.head()

(1058, 2)


,raceId,circuitId
0,1,1
1,2,2
2,3,17
3,4,3
4,5,4


## Join Tables

In [5]:
# Merge results with races on raceId.
data = (results.merge(races, left_on='raceId', right_on='raceId').reindex(
            columns=['year', 'round', 'resultId', 'raceId', 'circuitId', 'driverId', 
                     'constructorId','grid', 'position', 'weather_warm', 'weather_cold', 
                     'weather_dry', 'weather_wet', 'weather_cloudy' ]))
print(data.shape)
data.head()

(25040, 14)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0


In [6]:
print(data.shape)
data.head()

(25040, 14)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0


In [7]:
# Replace all "\\N" in position column to 0 
replace1 = {"\\N":0}
data["position"] = data["position"].replace(replace1)

# Convert 'position' from object to numeric
data['position'] = pd.to_numeric(data['position'])

## Other Transformations

In [8]:
# Add a new column titled "win". This column will map all wins to 1 and else 0 from "position" column
data['Win'] = data["position"]
data.Win = data.position.map(lambda x: 1 if x == 1 else 0)
print(data.shape)
data.head()

(25040, 15)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,Win
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0,1
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0,0
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0,0
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0,0


In [9]:
# Add a second new column titled "podium". 
#This column will map all podiums to 1,2,3 and else 0 from "position" column

podium = [1,2,3]

def podium_order(x):
    if x in podium:
        return x
    else:
        return 0
data["podium"] = data["position"].apply(podium_order)    
print(data.shape)
data.head()

(25040, 16)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,Win,podium
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0,1,1
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0,0,2
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0,0,3
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0,0,0


In [10]:
# We will filter the data for years 2000 to 2019

mldata = data[(data['year']<=2019) & (data['year']>=2000)]
print(mldata.shape)
mldata.head()

(7940, 16)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,Win,podium
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0,1,1
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0,0,2
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0,0,3
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0,0,0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0,0,0


## Calculate Driver Age

In [11]:
# Read in drivers.csv from database
cursor.execute("Select * FROM drivers")
data = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
drivers_df = pd.DataFrame(data, columns=colnames)

In [12]:
# Read in races.csv again. 
cursor.execute("Select * FROM races")
data = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
races_df = pd.DataFrame(data, columns=colnames)

In [13]:
results1 = results[['resultId', 'raceId', 'driverId']].copy()
races = races_df[['raceId','date']].copy()
drivers = drivers_df[['driverId', 'dob']].copy()

In [14]:
# Convert "date" from string to date in races dataframe
races['date'] = pd.to_datetime(races['date'])
races.dtypes

raceId             int64
date      datetime64[ns]
dtype: object

In [15]:
# Convert "dob" from string to date in drivers dataframe
drivers['dob'] = pd.to_datetime(drivers['dob'])
drivers.dtypes

driverId             int64
dob         datetime64[ns]
dtype: object

In [16]:
# Merge results1 with races on raceId.
agedata = (results1.merge(races, left_on='raceId', right_on='raceId').reindex(
            columns=['resultId', 'raceId', 'driverId', 'date']))

In [17]:
# Merge results1 with drivers on driverId
agedata = (agedata.merge(drivers, left_on='driverId', right_on='driverId').reindex(
            columns=['resultId', 'raceId', 'driverId', 'date', 'dob']))

In [18]:
# Create Age column by subtracting DOB from race date
agedata['age'] = (agedata['date'] - agedata['dob']) / np.timedelta64(1, 'Y')
agedata['age'] = agedata['age'].round(0)

In [19]:
# Sort dataframe by resultId in ascending order and drop the columns we dont need
agedata = agedata.sort_values(["resultId"], ascending = (True))
agedata = agedata.drop(['date', 'dob', 'raceId', 'driverId'], axis=1)
print(agedata.shape)
agedata.head()

(25040, 2)


,resultId,age
0,1,23.0
270,2,31.0
454,3,23.0
660,4,27.0
978,5,26.0


In [20]:
#Merge agedata with mldata
mldata = pd.merge(mldata, agedata,  how='left', left_on=['resultId'], right_on = ['resultId'])
print(mldata.shape)
mldata.head()

(7940, 17)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,Win,podium,age
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0,1,1,23.0
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0,0,2,31.0
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0,0,3,23.0
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0,0,0,27.0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0,0,0,26.0


## Driver standing before Race

In [21]:
# Read in driver_standings.csv from database
cursor.execute("Select * FROM driver_standings")
data = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
d_standing_raw = pd.DataFrame(data, columns=colnames)

In [22]:
# Merge d_standing_raw with races_df on raceId.
d_rawpoints = (d_standing_raw.merge(races_df, left_on='raceId', right_on='raceId').reindex(
            columns=['driverStandingsId', 'raceId', 'year', 'round', 'driverId', 'position', 'wins']))
print(d_rawpoints.shape)
d_rawpoints.head()

(33025, 7)


,driverStandingsId,raceId,year,round,driverId,position,wins
0,1,18,2008,1,1,1,1
1,2,18,2008,1,2,2,0
2,3,18,2008,1,3,3,0
3,4,18,2008,1,4,4,0
4,5,18,2008,1,5,5,0


In [23]:
# Create a copy of rawpoints df
d_editpoints = d_rawpoints.copy()

The 'position' and 'wins' columns represent league position and wins made after the race. However we want to create a table which shows the position and wins before the start of the race. In order to do so, we will add a +1 to the raceId and create a new dataframe called 'editpoints'. Essentially the new dataframe will have the league standings  and wins made for that season as of the start of the race. We will then merge this with the old 'rawpoints'using raceId and driverId and create a new dataframe which has the points earned and races won for each driver as of the start of the race for that season. The only exception to that will be round 1 which will have the wins made and league standings from the last race of the previous season (since we added a +1 to each raceId). We will fix this issue by making points earned and races won for each round 1 race equal to 0. 

In [24]:
# Add a +1 to each raceId.  
d_editpoints['raceId'] = d_editpoints['raceId']+1

In [25]:
# Drop unnecessary columns which we don't need. 
d_editpoints = d_editpoints.drop(['driverStandingsId', 'year', 'round'], axis=1)
d_editpoints.head()

,raceId,driverId,position,wins
0,19,1,1,1
1,19,2,2,0
2,19,3,3,0
3,19,4,4,0
4,19,5,5,0


In [26]:
# Merge rawppoints and editpoints on raceId and driverId
dpoints = pd.merge(d_rawpoints, d_editpoints,  how='left', left_on=['raceId','driverId'], right_on = ['raceId','driverId'])

In [27]:
# Turn the points earned and races won for Round 1 to 0 wins and 0 points
dpoints.loc[(dpoints["round"]==1),"position_y"] = 0
dpoints.loc[(dpoints["round"]==1),"wins_y"] = 0

In [28]:
# Drop columns we don't need
dpoints = dpoints.drop(['driverStandingsId', 'position_x', 'wins_x'], axis=1)
# Rename columns
dpoints = dpoints.rename(columns={'position_y': 'driverposition', 'wins_y':'driverwin'})

In [29]:
# Filter points dataframe to only include data from 2000 to 2019
dmlpoints = dpoints[(dpoints['year']<=2019) & (dpoints['year']>=2000)].copy()

In [30]:
# replace null values in 'driverposition' and 'driverwin' columns with zeros
dmlpoints[['driverposition', 'driverwin']] = dmlpoints[['driverposition', 'driverwin']].fillna(0)

In [31]:
# Double check if there are any further null values
for i in dmlpoints.columns:
    print(f"Columns {i} has {dmlpoints[i].isnull().sum()} null values")

Columns raceId has 0 null values
Columns year has 0 null values
Columns round has 0 null values
Columns driverId has 0 null values
Columns driverposition has 0 null values
Columns driverwin has 0 null values


In [32]:
# Drop year and round columns as we don't need them anymore. This will help with the merge with mldata
dmlpoints = dmlpoints.drop(['year', 'round'], axis=1)

In [33]:
# Print final dataframe
print(dmlpoints.shape)
dmlpoints.columns

(8255, 4)


Index(['raceId', 'driverId', 'driverposition', 'driverwin'], dtype='object')

In [34]:
#Merge mldata with dmlpoints
mldata = pd.merge(mldata, dmlpoints,  how='left', left_on=['raceId', 'driverId'], right_on = ['raceId', 'driverId'])
print(mldata.shape)
mldata.head()

(7940, 19)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,Win,podium,age,driverposition,driverwin
0,2008,1,1,18,1,1,1,1,1,1,0,0,0,0,1,1,23.0,0.0,0.0
1,2008,1,2,18,1,2,2,5,2,1,0,0,0,0,0,2,31.0,0.0,0.0
2,2008,1,3,18,1,3,3,7,3,1,0,0,0,0,0,3,23.0,0.0,0.0
3,2008,1,4,18,1,4,4,11,4,1,0,0,0,0,0,0,27.0,0.0,0.0
4,2008,1,5,18,1,5,1,3,5,1,0,0,0,0,0,0,26.0,0.0,0.0


## Constructor standing before Race

In [35]:
# Read in constructor_standings.csv from database
cursor.execute("Select * FROM constructor_standings")
data = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
c_standing_raw = pd.DataFrame(data, columns=colnames)

In [36]:
# Merge c_standings with races_df on raceId.
c_rawpoints = (c_standing_raw.merge(races_df, left_on='raceId', right_on='raceId').reindex(
            columns=['constructorStandingsId', 'raceId', 'year', 'round', 'constructorId', 'position', 'wins']))
print(c_rawpoints.shape)
c_rawpoints.head()

(12536, 7)


,constructorStandingsId,raceId,year,round,constructorId,position,wins
0,1,18,2008,1,1,1,1
1,2,18,2008,1,2,3,0
2,3,18,2008,1,3,2,0
3,4,18,2008,1,4,4,0
4,5,18,2008,1,5,5,0


In [37]:
# Create a copy of rawpoints df
c_editpoints = c_rawpoints.copy()

The 'position' and 'wins' columns represent points earned and wins made after the race. However we want to create a table which shows the position and wins before the start of the race. In order to do so, we will add a +1 to the raceId and create a new dataframe called 'editpoints'. Essentially the new dataframe will have the position  and wins made for that season as of the start of the race. We will then merge this with the old 'rawpoints'using raceId and constructorId and create a new dataframe which has the league standings and races won for each constructor as of the start of the race for that season. The only exception to that will be round 1 which will have the wins made and league standings from the last race of the previous season (since we added a +1 to each raceId). We will fix this issue by making league standings and races won for each round 1 race equal to 0. 

In [38]:
# Add a +1 to each raceId.  
c_editpoints['raceId'] = c_editpoints['raceId']+1

In [39]:
# Drop unnecessary columns which we don't need. 
c_editpoints = c_editpoints.drop(['constructorStandingsId', 'year', 'round'], axis=1)
c_editpoints.head()

,raceId,constructorId,position,wins
0,19,1,1,1
1,19,2,3,0
2,19,3,2,0
3,19,4,4,0
4,19,5,5,0


In [40]:
# Merge c_rawppoints and c_editpoints on raceId and driverId
cpoints = pd.merge(c_rawpoints, c_editpoints,  how='left', left_on=['raceId','constructorId'], right_on = ['raceId','constructorId'])

In [41]:
# Turn the points earned and races won for Round 1 to 0 wins and 0 points
cpoints.loc[(cpoints["round"]==1),"position_y"] = 0
cpoints.loc[(cpoints["round"]==1),"wins_y"] = 0

In [42]:
# Drop columns we don't need
cpoints = cpoints.drop(['constructorStandingsId', 'position_x', 'wins_x'], axis=1)
# Rename columns
cpoints = cpoints.rename(columns={'position_y': 'constructorposition', 'wins_y':'constructorwin'})

In [43]:
# Filter points dataframe to only include data from 2000 to 2019
cmlpoints = cpoints[(cpoints['year']<=2019) & (cpoints['year']>=2000)].copy()

In [44]:
# replace null values in 'driverposition' and 'driverwin' columns with zeros
cmlpoints[['constructorposition', 'constructorwin']] = cmlpoints[['constructorposition', 'constructorwin']].fillna(0)

In [45]:
# Double check if there are any further null values
for i in cmlpoints.columns:
    print(f"Columns {i} has {cmlpoints[i].isnull().sum()} null values")

Columns raceId has 0 null values
Columns year has 0 null values
Columns round has 0 null values
Columns constructorId has 0 null values
Columns constructorposition has 0 null values
Columns constructorwin has 0 null values


In [46]:
# Drop year and round columns as we don't need them anymore. This will help with the merge with mldata
cmlpoints = cmlpoints.drop(['year', 'round'], axis=1)

In [47]:
# Print final dataframe
print(cmlpoints.shape)
cmlpoints.columns

(3969, 4)


Index(['raceId', 'constructorId', 'constructorposition', 'constructorwin'], dtype='object')

In [48]:
#Merge mldata with cmlpoints
mldata = pd.merge(mldata, cmlpoints,  how='left', left_on=['raceId', 'constructorId'], right_on = ['raceId', 'constructorId'])
print(mldata.shape)
mldata.head()

(7940, 21)


,year,round,resultId,raceId,circuitId,driverId,constructorId,grid,position,weather_warm,...,weather_dry,weather_wet,weather_cloudy,Win,podium,age,driverposition,driverwin,constructorposition,constructorwin
0,2008,1,1,18,1,1,1,1,1,1,...,0,0,0,1,1,23.0,0.0,0.0,0.0,0.0
1,2008,1,2,18,1,2,2,5,2,1,...,0,0,0,0,2,31.0,0.0,0.0,0.0,0.0
2,2008,1,3,18,1,3,3,7,3,1,...,0,0,0,0,3,23.0,0.0,0.0,0.0,0.0
3,2008,1,4,18,1,4,4,11,4,1,...,0,0,0,0,0,27.0,0.0,0.0,0.0,0.0
4,2008,1,5,18,1,5,1,3,5,1,...,0,0,0,0,0,26.0,0.0,0.0,0.0,0.0


## Export mldata to database

In [51]:
# We will upload our final mldata back into our cloud Postgress database. Create engine to upload data
connect = f"postgres://postgres:{PASSWORD}@{HOST}:{PORT}/postgres"
from sqlalchemy import create_engine
engine = create_engine(connect)

In [52]:
# Upload mldata table to database
mldata.to_sql(name='mldata', con=engine, if_exists='replace')